In [20]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageDraw, ImageOps
import os
import datetime
import numpy as np
import math
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import logging

In [21]:
# === CONFIGURACIÓN GENERAL ===
CANVAS_WIDTH = 200
CANVAS_HEIGHT = 200
BRUSH_SIZE = 12
OUTPUT_IMAGE_NAME = "numero.png"
OUTPUT_FOLDER = "imagenes"
OUTPUT_SIZE = (28, 28)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [22]:
# === ENTRENAR EL MODELO ===
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

dataset, metadata = tfds.load('mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']
class_names = ['Cero', 'Uno', 'Dos', 'Tres', 'Cuatro', 'Cinco', 'Seis', 'Siete', 'Ocho', 'Nueve']
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples

In [23]:
def normalize(images, labels):
    images = tf.cast(images, tf.float32)
    images /= 255
    return images, labels

train_dataset = train_dataset.map(normalize)
test_dataset = test_dataset.map(normalize)

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [24]:
BATCHSIZE = 32
train_dataset = train_dataset.repeat().shuffle(num_train_examples).batch(BATCHSIZE)
test_dataset = test_dataset.batch(BATCHSIZE)

In [25]:
print("Entrenando modelo...")
model.fit(train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples / BATCHSIZE))
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples / 32))
print("Precisión en pruebas:", test_accuracy)

Entrenando modelo...
Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8637 - loss: 0.4828
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9587 - loss: 0.1388
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9733 - loss: 0.0881
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9765 - loss: 0.0782
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9808 - loss: 0.0620
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9718 - loss: 0.0973
Precisión en pruebas: 0.9713000059127808


In [26]:
# === CLASE DE TKINTER PARA DIBUJAR Y PREDECIR ===
class Pizarron:
    def __init__(self, master):
        self.master = master
        self.master.title("Dibuja un número")
        self.master.geometry(f"{CANVAS_WIDTH+20}x{CANVAS_HEIGHT+20}")

        self.canvas = tk.Canvas(master, width=CANVAS_WIDTH, height=CANVAS_HEIGHT, bg='white')
        self.canvas.pack(padx=10, pady=10)

        self.image = Image.new("L", (CANVAS_WIDTH, CANVAS_HEIGHT), 255)
        self.draw = ImageDraw.Draw(self.image)

        self.canvas.bind("<B1-Motion>", self.dibujar)
        self.canvas.bind("<ButtonRelease-1>", self.guardar_y_predecir)

    def dibujar(self, event):
        x, y = event.x, event.y
        x1, y1 = x - BRUSH_SIZE // 2, y - BRUSH_SIZE // 2
        x2, y2 = x + BRUSH_SIZE // 2, y + BRUSH_SIZE // 2
        self.canvas.create_oval(x1, y1, x2, y2, fill='black', outline='black')
        self.draw.ellipse([x1, y1, x2, y2], fill=0)

    def guardar_y_predecir(self, event):
        # Redimensionar a 28x28
        resized = self.image.resize(OUTPUT_SIZE, Image.LANCZOS)
        final_path = os.path.join(OUTPUT_FOLDER, OUTPUT_IMAGE_NAME)
        resized.save(final_path)

        # Mostrar mensaje
        messagebox.showinfo("Guardado", f"Imagen guardada como {OUTPUT_IMAGE_NAME}")
        print("Imagen guardada como", OUTPUT_IMAGE_NAME)

        # Predecir
        self.predecir_imagen(final_path)

        # Limpiar canvas
        self.limpiar_canvas()

    def predecir_imagen(self, image_path):
        img = Image.open(image_path).convert("L")
        img_array = np.array(img).reshape((1, 28, 28, 1)).astype("float32") / 255.0
        prediction = model.predict(img_array)
        predicted_label = np.argmax(prediction)

        print(f"Predicción: {class_names[predicted_label]} ({predicted_label})")

    def limpiar_canvas(self):
        self.canvas.delete("all")
        self.image = Image.new("L", (CANVAS_WIDTH, CANVAS_HEIGHT), 255)
        self.draw = ImageDraw.Draw(self.image)

In [27]:
root = tk.Tk()
app = Pizarron(root)
root.mainloop()

Imagen guardada como numero.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
Predicción: Cinco (5)
Imagen guardada como numero.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Predicción: Cinco (5)
